In [ ]:
!mkdir data

In [9]:
!wget "http://localhost:8080/SnomedDLQuery?message=105590001" -O "data/substances.json"

--2023-06-27 07:53:35--  http://localhost:8080/SnomedDLQuery?message=105590001
Resolving localhost (localhost)... 127.0.0.1
Connecting to localhost (localhost)|127.0.0.1|:8080... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/json]
Saving to: ‘data/substances.json’

data/substances.jso     [ <=>                ]   3.12M  --.-KB/s    in 0.02s   

2023-06-27 07:53:36 (168 MB/s) - ‘data/substances.json’ saved [3275901]



In [10]:
!wget "http://localhost:8080/SnomedDLQuery?message=736479009" -O "data/roa.json"

--2023-06-27 07:53:39--  http://localhost:8080/SnomedDLQuery?message=736479009
Resolving localhost (localhost)... 127.0.0.1
Connecting to localhost (localhost)|127.0.0.1|:8080... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/json]
Saving to: ‘data/roa.json’

data/roa.json           [ <=>                ]   5.68K  --.-KB/s    in 0s      

2023-06-27 07:53:40 (308 MB/s) - ‘data/roa.json’ saved [5817]



In [11]:
!wget "http://localhost:8080/SnomedDLQuery?message=736542009" -O "data/form.json"

--2023-06-27 07:53:40--  http://localhost:8080/SnomedDLQuery?message=736542009
Resolving localhost (localhost)... 127.0.0.1
Connecting to localhost (localhost)|127.0.0.1|:8080... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/json]
Saving to: ‘data/form.json’

data/form.json          [ <=>                ]  77.16K  --.-KB/s    in 0s      

2023-06-27 07:53:41 (213 MB/s) - ‘data/form.json’ saved [79010]



In [12]:
 !wget "http://localhost:8080/SnomedDLQuery?message=284009009" -O "data/roa_two.json"

--2023-06-27 07:53:41--  http://localhost:8080/SnomedDLQuery?message=284009009
Resolving localhost (localhost)... 127.0.0.1
Connecting to localhost (localhost)|127.0.0.1|:8080... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/json]
Saving to: ‘data/roa_two.json’

data/roa_two.json       [ <=>                ]  31.57K  --.-KB/s    in 0s      

2023-06-27 07:53:42 (222 MB/s) - ‘data/roa_two.json’ saved [32328]



In [13]:
 !wget "http://localhost:8080/SnomedDLQuery?message=763158003" -O "data/medical_product.json"

--2023-06-27 07:53:42--  http://localhost:8080/SnomedDLQuery?message=763158003
Resolving localhost (localhost)... 127.0.0.1
Connecting to localhost (localhost)|127.0.0.1|:8080... connected.
HTTP request sent, awaiting response... 200 
Length: unspecified [application/json]
Saving to: ‘data/medical_product.json’

data/medical_produc     [ <=>                ]   4.19M  --.-KB/s    in 0.02s   

2023-06-27 07:53:43 (212 MB/s) - ‘data/medical_product.json’ saved [4390514]



In [1]:
import json
import os
import ast
import pandas as pd

def load_wget_results():
    def helper(filename):
        fullpath = f"data/{filename}"
        with open(fullpath,"r") as f:
            content = f.read()[1:-1]
#             import ipdb; ipdb.set_trace()
            obj = ast.literal_eval(content)
            return pd.DataFrame(obj)
    return {filename: helper(filename) for filename in os.listdir("data") if filename.endswith(".json")}
   

In [2]:
dfs = load_wget_results()

In [3]:
type('hi') == str

True

In [4]:
dfs['medical_product.json'].dropna()

,iri,prefLabel,label,altLabel
48,874866000,Abacavir (as abacavir sulfate) 60 mg tablet fo...,Product containing precisely abacavir (as abac...,Abacavir (as abacavir sulfate) 60 mg dispersib...
61,255683006,Immunotherapeutic agent,Immunotherapeutic agent (product),Immunologic agent
145,1119349007,SARS-CoV-2 mRNA vaccine,Vaccine product containing only severe acute r...,Severe acute respiratory syndrome coronavirus ...
230,2251000221101,Live attenuated Measles morbillivirus and Mump...,Vaccine product containing only live attenuate...,Live attenuated measles and mumps and rubella ...
591,2071000221100,Whole cell Bordetella pertussis and Corynebact...,Pediatric vaccine product containing only whol...,Diphtheria toxoid and Haemophilus influenzae t...
...,...,...,...,...
23756,770311004,Cromolyn sodium 5 mg/actuation pressurized sus...,Product containing precisely cromoglicate sodi...,Sodium cromoglicate 5 mg/actuation pressurized...
23793,1156183006,Human alphaherpesvirus 3 recombinant surface g...,Vaccine product containing only Human alphaher...,Recombinant Varicella-zoster glycoprotein E va...
23828,871744002,Cytomegalovirus antibody only product in paren...,Product containing only Cytomegalovirus antibo...,Cytomegalovirus immunoglobulin only product in...
23836,353844008,Radionuclide generators,Radionuclide generators (product),Radionuclide generator


In [5]:
def find_clinical_drugs(meds_df):
    mask = [type(med) == str and '(clinical drug)' in med for med in meds_df['label'].values]
    return meds_df.loc[mask]

In [6]:
drug_df = find_clinical_drugs(dfs['medical_product.json'])

In [7]:
def export_drug(drug_df):
    new_df = drug_df[['iri','prefLabel']].copy()
    new_df.columns = ['snomed_code','snomed_label']
    new_df.to_csv('export/snomed_drug.csv')
export_drug(drug_df)

In [8]:
list(dfs['medical_product.json']['label'].values)

['Product containing precisely chlorpropamide 250 milligram/1 each conventional release oral tablet (clinical drug)',
 'Product containing precisely dactinomycin 500 microgram/1 vial powder for conventional release solution for injection (clinical drug)',
 'Product containing gemcitabine (medicinal product)',
 'Product containing potassium perchlorate in oral dose form (medicinal product form)',
 'Product containing precisely clotrimazole 100 milligram/1 gram conventional release vaginal cream (clinical drug)',
 'Product containing precisely phenol 20 milligram/1 milliliter conventional release cutaneous solution (clinical drug)',
 'Product containing only halothane (medicinal product)',
 'Product containing tiopronin in oral dose form (medicinal product form)',
 'Product containing only torasemide in oral dose form (medicinal product form)',
 'Product containing dexmethylphenidate and serdexmethylphenidate (medicinal product)',
 'Product containing glucose and potassium chloride and s

In [9]:
dfs['form.json']

,iri,prefLabel,altLabel,label
0,385105007,Cutaneous spray,Conventional release cutaneous spray,Conventional release cutaneous spray (dose form)
1,1231323007,Powder for dental cement,Conventional release powder for dental cement,Conventional release powder for dental cement ...
2,422353003,Prolonged-release oral suspension,Extended-release oral suspension,Prolonged-release oral suspension (dose form)
3,1163573008,Film-coated oral tablet,Conventional release film-coated oral tablet,Conventional release film-coated oral tablet (...
4,385128008,Prolonged-release eye drops,Extended-release eye drops,Prolonged-release eye drops (dose form)
...,...,...,...,...
442,385162008,Nasal stick,Conventional release nasal stick,Conventional release nasal stick (dose form)
443,1229992005,Powder for solution for iontophoresis,Powder for conventional release solution for i...,Powder for conventional release solution for i...
444,1231563007,Powder for vapor inhalation,Conventional release powder for vapour inhalation,Conventional release powder for vapor inhalati...
445,1255299001,Tablet for oromucosal solution for mouthwash,Tablet for conventional release oromucosal sol...,Tablet for conventional release oromucosal sol...


In [10]:
dfs['form.json'][['iri','prefLabel']]

,iri,prefLabel
0,385105007,Cutaneous spray
1,1231323007,Powder for dental cement
2,422353003,Prolonged-release oral suspension
3,1163573008,Film-coated oral tablet
4,385128008,Prolonged-release eye drops
...,...,...
442,385162008,Nasal stick
443,1229992005,Powder for solution for iontophoresis
444,1231563007,Powder for vapor inhalation
445,1255299001,Tablet for oromucosal solution for mouthwash


In [11]:
dfs['roa.json'][['iri','prefLabel']]

,iri,prefLabel
0,738946006,Intravesical
1,738948007,Nasal
2,1255345008,Intestinal
3,762070008,Intraocular
4,738952007,Ocular
5,764295003,Subcutaneous
6,738984000,Parenteral
7,738988002,Urethral
8,761829007,Sublingual
9,764786007,Oropharyngeal


In [12]:
dfs['roa_two.json']

,iri,prefLabel,definition,label,altLabel
0,447227007,Transtympanic route,An otic route that begins across the tympanic ...,Transtympanic route (qualifier value),NaN
1,60213007,Intramedullary route,An intraosseous route that begins within the m...,Intramedullary route (qualifier value),Intramedullary use
2,766790006,Extracorporeal haemodialysis route,A route used for extracorporeal hemodialysis w...,Extracorporeal hemodialysis route (qualifier v...,NaN
3,38239002,Intraperitoneal route,A body cavity route that begins within the per...,Intraperitoneal route (qualifier value),Intraperitoneal use
4,418287000,Intracoronal route,A dental route that begins within a portion of...,Intracoronal route (qualifier value),NaN
...,...,...,...,...,...
159,372471009,Intravesical route,A body cavity route that begins within the bla...,Intravesical route (qualifier value),Intravesical use
160,714743009,Extracorporeal route,A route that occurs outside of the body.,Extracorporeal route (qualifier value),NaN
161,372454008,Gastroenteral route,A digestive tract route that begins in the gas...,Gastroenteral route (qualifier value),Gastroenteral use
162,447964005,Digestive tract route,A route that begins anywhere in the digestive ...,Digestive tract route (qualifier value),NaN


In [13]:
def clean_df(df):
    new_df = df[['iri','prefLabel']].copy()
    new_df['prefLabel'] = new_df['prefLabel'].str.upper()
    new_df.columns = ['snomed_code','snomed_label']
    return new_df.dropna()

def clean_form_snomed(raw_form_df):
    mapping = {
        "EYE" : "OPHTALMIC",
        "EAR" : "OTIC",
        "NOSE" : "NASAL"
    }
    def sub_form_snomed_helper(form):
        return ' '.join([mapping.get(el,el) for el in form.split(" ")])
    form_df = raw_form_df.copy()
    form_df['old_snomed_label'] = form_df['snomed_label'].copy()
    form_df['snomed_label'] = [sub_form_snomed_helper(el.replace(" DOSE FORM","")) for el in form_df['old_snomed_label'].values]
    return form_df

def clean_form_ocrx(raw_form_df):
    def clean_form_ocrx_helper(el):
        # removes the word in and inverts it
        if " IN " in el:
            splitted = el.split(" IN ")
            if len(splitted) > 2:
                import ipdb; ipdb.set_trace()
            start, last = splitted[:2]
            return last + " " + start
        else:
            return el
            
    form_df = raw_form_df.copy()
#     import ipdb; ipdb.set_trace()
    form_df['old_ocrx_label'] = form_df['ocrx_label'].copy()
    form_df['ocrx_label'] = [clean_form_ocrx_helper(el) for el in form_df['old_ocrx_label'].values]
    return form_df

In [11]:
clean_dfs = {key: clean_df(df) for key, df in dfs.items()}

In [12]:
clean_dfs['substances.json']

,snomed_code,snomed_label
0,47974007,BLOOD GROUP ANTIBODY TG^A^
1,24851008,DEOXYRIBONUCLEIC ACID
2,707141001,2-HYDROXYCAPROATE
3,387940006,ITALIAN CYPRESS SPECIFIC IMMUNOGLOBULIN E
4,129994000,QUINATE DEHYDROGENASE (PYRROLOQUINOLINE-QUINONE)
...,...,...
27187,396462008,BAMBUTEROL
27188,54791001,METANIL YELLOW STAIN
27189,418540006,CALCIUM REGULATING AGENT
27190,49069005,LEAD ISOTOPE


In [15]:
clean_dfs['drug.json']

NameError: name 'clean_dfs' is not defined

In [13]:
def gather_ocrx_medical_term(medical_term):
    df = pd.read_csv(f"/home/james/Datasets/ocrx-owls/match-ocrx/match_{medical_term}_en.csv")
    df = df.drop_duplicates().sort_values("sLabel")
    labels = df['sLabel'].values
    codes = df['s'].values
    return [(str(val),str(val_code)) for val, val_code in zip(labels,codes)]

def parse_dosage(dose):
    splitted = dose.split(" ")
    unit = splitted[-1]
    if unit in units_conversion:
        pass
# print('\n'.join([str(el) for el in df_compo['dose'].unique()]))
ocrx_terms = ['form','roa','substance']
ocrx_dfs = [gather_ocrx_medical_term(term) for term in ocrx_terms]

def convert_ocrx_to_df(ocrx_elems):
    return [pd.DataFrame(ocrx_df,columns=['ocrx_label','ocrx_code']) for ocrx_df in ocrx_elems]

ocrx_dfs = convert_ocrx_to_df(ocrx_dfs)
ocrx_dfs[0] = clean_form_ocrx(ocrx_dfs[0])
ocrx_form_df, ocrx_roa_df, ocrx_substance_df = ocrx_dfs

from pytrie import Trie
def make_trie(ocrx_def_df):
    return {row['ocrx_label']: row['ocrx_code'] for _, row in ocrx_def_df.iterrows()}

def make_trie_ansm(ansm_def_df):
    return {row['snomed_label']: row['snomed_code'] for _, row in ansm_def_df.iterrows()}

def get_prefix(trie,term):
    words = term.split(' ')
    for i in reversed(range(1,len(words)+1)):
        term = ' '.join(words[:i])
        if term in trie:
            return (term, trie[term])
    return None


snomed_dfs = [clean_form_snomed(clean_dfs['form.json']),clean_dfs['roa.json'],clean_dfs['substances.json']]
ocrx_form_trie,ocrx_roa_trie,ocrx_substance_trie = [make_trie(el) for el in ocrx_dfs]
snomed_tries = [make_trie_ansm(el) for el in snomed_dfs]

In [14]:
import os
DIR = 'match-ocrx-full'
def print_stats(df_merged,type_fn,name='snomed'):
    # total_match
    prefix_code = f"{name}_code"
    print("TOTAL MATCH:")
    total_df = df_merged.dropna()
    print(total_df.head())
    print(total_df.shape)
    total_df.to_csv(f"{DIR}/match_{type_fn}_total_match.csv")
    # left missing
    print("ONLY OCRX")
    only_ocrx = df_merged.loc[df_merged[prefix_code].isna()]
    print(only_ocrx.head())
    print(only_ocrx.shape)
    only_ocrx.to_csv(f"{DIR}/match_{type_fn}_only_ocrx.csv")
    # right missing
    print(f"ONLY {name.upper()}")
    only_ansm = df_merged.loc[df_merged['ocrx_code'].isna()]
    print(only_ansm.head())
    print(only_ansm.shape)
    stats = {'total' : total_df.shape[0], 'ocrx' : only_ocrx.shape[0], name : only_ansm.shape[0]}
    only_ansm.to_csv(f"{DIR}/match_{type_fn}_only_{name}.csv")
    return stats

def add_prefixes(df,trie,trie_ansm,prefix='snomed'):
    prefix_code = f"{prefix}_code"
    prefix_label = f"{prefix}_label"
    def process_row(row,trie):
        if type(row['ocrx_code']) == str and type(row[prefix_code]) == str:
            return (row['ocrx_code'], row['ocrx_label'], 'EXACT')
        elif str(row['ocrx_code']) == 'nan':
            result = get_prefix(trie,row[prefix_label])
            if result is None:
                return (None, None, 'NONE')
            else:
                prefix_ocrx, ocrx_code = result
                return (ocrx_code, prefix_ocrx, 'PREFIX')
        else:
            return (row['ocrx_code'], row['ocrx_label'], 'NA')
    def process_row_ansm(row,trie):
        if type(row['ocrx_code']) == str and type(row[prefix_code]) == str:
            return (row[prefix_code], row[prefix_label], 'EXACT')
        elif str(row[prefix_code]) == 'nan':
            result = get_prefix(trie,row['ocrx_label'])
            if result is None:
                return (None, None, 'NONE')
            else:
                prefix_ansm, ansm_code = result
                return (ansm_code, prefix_ansm, 'PREFIX')
        else:
            return (row[prefix_code], row[prefix_label], 'NA')
        
            
#     import ipdb; ipdb.set_trace()
    els = ['ocrx','snomed']
    new_df = df.copy()
    for el, trie_el, process_fn in zip(els,[trie,trie_ansm],[process_row,process_row_ansm]):
        results = [process_fn(row,trie_el) for _, row in df.iterrows()]
        codes = [result[0] for result in results]
        full_terms = [result[1] for result in results]
        match_types = [result[2] for result in results]
        new_df[f'{el}_code'] = codes
        new_df[f'{el}_label'] = full_terms
        new_df[f'{el}_match_type'] = match_types
    return new_df

filenames = ['form','roa','substance']
if not os.path.exists(DIR):
    os.mkdir(DIR)
ocrxs = [ocrx_form_df,ocrx_roa_df,ocrx_substance_df]
# ansms = [bdpm_form_df,bdpm_roa_df,bdpm_substance_df]
others = snomed_dfs
others_tries = snomed_tries
tries = [ocrx_form_trie,ocrx_roa_trie,ocrx_substance_trie]
all_stats = dict()
for ocrx, ansm, filename, trie, trie_ansm in zip(ocrxs,others,filenames,tries,others_tries):
    fullpath = f"{DIR}/match_{filename}.csv"
    df_merged = pd.merge(ocrx,ansm,how="outer",left_on=['ocrx_label'], right_on=['snomed_label'])
#     import ipdb; ipdb.set_trace()
    df_merged_with_prefix = add_prefixes(df_merged,trie,trie_ansm)
    all_stats[filename] = print_stats(df_merged_with_prefix,filename)
    df_merged_with_prefix.to_csv(fullpath)

TOTAL MATCH:
          ocrx_label                           ocrx_code      old_ocrx_label  \
104     ORAL CAPSULE  http://www.ocrx.ca/OCRx/4000000461     CAPSULE IN ORAL   
105     ORAL CAPSULE  http://www.ocrx.ca/OCRx/4000000461     CAPSULE IN ORAL   
112  VAGINAL CAPSULE  http://www.ocrx.ca/OCRx/4000000462  CAPSULE IN VAGINAL   
113  VAGINAL CAPSULE  http://www.ocrx.ca/OCRx/4000000462  CAPSULE IN VAGINAL   
134    VAGINAL CREAM  http://www.ocrx.ca/OCRx/4000000693    CREAM IN VAGINAL   

    snomed_code     snomed_label old_snomed_label ocrx_match_type  \
104   420692007     ORAL CAPSULE     ORAL CAPSULE           EXACT   
105   420692007     ORAL CAPSULE     ORAL CAPSULE           EXACT   
112   385175008  VAGINAL CAPSULE  VAGINAL CAPSULE           EXACT   
113   385175008  VAGINAL CAPSULE  VAGINAL CAPSULE           EXACT   
134   385165005    VAGINAL CREAM    VAGINAL CREAM           EXACT   

    snomed_match_type  
104             EXACT  
105             EXACT  
112             EXA

In [15]:
all_stats

{'form': {'total': 124, 'ocrx': 1283, 'snomed': 263},
 'roa': {'total': 18, 'ocrx': 80, 'snomed': 14},
 'substance': {'total': 746485, 'ocrx': 2085980, 'snomed': 21076}}

In [41]:
import re
def clean_dose(drug):
    unit_map = {
        'MICROGRAM' : 10 ** -3,
        'MG' : 1
    }
    numb_patt = re.compile(r'[0-9]+')
    splitted = drug.split(' ')
    resul_splitted = drug.split(' ')
    dose_indices = []
    for i, tok in enumerate(splitted):
        if numb_patt.sub('',tok) == '' and i > 0:
            unit_ref = splitted[i+1]
            first_quant = unit_ref.split('/')[0]
            if first_quant in unit_map:
                multiplier = unit_map[first_quant]
                numb = float(tok) * multiplier
                if numb == int(numb):
                    numb = int(numb)
                resul_splitted[i] = str(numb)
                resul_splitted[i+1] = 'MG' + unit_ref[len(first_quant):]
    return ' '.join(resul_splitted)

def remove_ands(drug):
    return ' | '.join(drug.split(' AND '))

def remove_second_in(drug):
    if len(drug.split(' FOR ')) == 3:
        a,b,c = drug.split(' FOR ')
        return a + ' FOR ' +  b + ' IN ' + c
    else:
        return drug
def clean_drug_snomed(drug):
    extra_paren_patt = re.compile(r'\(AS [^)]+\) ')
    clean_drug = extra_paren_patt.sub('',drug)
    standard_dose_drug = clean_dose(clean_drug)
    no_ands_standard_drug = remove_ands(standard_dose_drug)
    no_ands_no_in_standard_drug = remove_second_in(no_ands_standard_drug)
    return no_ands_no_in_standard_drug
count = -1
step = 0
processed = drug_df['prefLabel'].str.upper().apply(clean_drug_snomed).values[step*count:step*count+count]
original = drug_df['prefLabel'].str.upper().values[step*count:step*count+count]
# for orig, proc in zip(original,processed):
#     if len(orig.split(' FOR ')) < 3:
#         continue
#     print(f"Orig: {orig}")
#     print(f"Proc: {proc}")
#     print('-------------')
    

new_df = drug_df.copy()
new_df['prefLabel'] = drug_df['prefLabel'].str.upper().apply(clean_drug_snomed)
export_drug(new_df)

In [42]:
new_df['prefLabel'].values

array(['CHLORPROPAMIDE 250 MG ORAL TABLET',
       'DACTINOMYCIN 0.5 MG POWDER FOR SOLUTION IN INJECTION VIAL',
       'CLOTRIMAZOLE 100 MG/G VAGINAL CREAM', ...,
       'TRIFLUOPERAZINE 2 MG PROLONGED-RELEASE ORAL CAPSULE',
       'MUCIN 35 MG/ML OROMUCOSAL SPRAY',
       'AMLODIPINE 5 MG | HYDROCHLOROTHIAZIDE 25 MG | OLMESARTAN MEDOXOMIL 40 MG ORAL TABLET'],
      dtype=object)

In [ ]:
!nemo .


** (nemo:1207004): WARNING **: 03:07:56.174: Current gtk theme is not known to have nemo support (Nordic-master) - checking...
ImportError: could not import gobject (error was: ModuleNotFoundError("No module named 'gi'"))

** (nemo:1207004): WARNING **: 03:07:56.423: pygobject initialization failed

** (nemo:1207004): WARNING **: 03:07:56.423: nemo_python_init_python failed
Nemo-Share-Message: 03:07:57.149: Called "net usershare info" but it failed: 'net usershare' returned error 255: net usershare: cannot open usershare directory /var/lib/samba/usershares. Error No such file or directory
Please ask your system administrator to enable user sharing.


** (nemo:1207004): WARNING **: 03:08:15.069: Could not inhibit power management: GDBus.Error:org.freedesktop.DBus.Error.NameHasNoOwner: Name "org.gnome.SessionManager" does not exist

(nemo:1207004): GLib-GIO-CRITICAL **: 03:08:22.204: g_file_get_parse_name: assertion 'G_IS_FILE (file)' failed

(nemo:1207004): GLib-GIO-CRITICAL **: 03:08: